In [1]:
import os
import glob
import json

import h5py
import scipy
import scipy.io as io
from scipy.ndimage import gaussian_filter

from PIL import Image
import numpy as np

import torch
from matplotlib import pyplot as plt
from matplotlib import cm as CM

import torchvision.transforms.functional as F

# only rely on modules that ship with this repo to avoid optional deps
from model import CSRNet

%matplotlib inline

In [2]:
from torchvision import datasets, transforms
transform=transforms.Compose([
                       transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
                   ])

In [3]:
root = '/root/autodl-tmp/Shanghai'

In [4]:
#now generate the ShanghaiA's ground truth
part_A_train = os.path.join(root,'part_A_final/train_data','images')
part_A_test = os.path.join(root,'part_A_final/test_data','images')
part_B_train = os.path.join(root,'part_B_final/train_data','images')
part_B_test = os.path.join(root,'part_B_final/test_data','images')
path_sets = [part_A_test]

In [5]:
img_paths = []
for path in path_sets:
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
        img_paths.append(img_path)

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CSRNet().to(device)
model.eval()

/root/miniconda3/envs/crowd/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/miniconda3/envs/crowd/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


CSRNet(
  (frontend): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilatio

In [ ]:
# model already moved to the chosen device above

In [7]:
checkpoint = torch.load('0model_best.pth.tar', map_location=device)

In [8]:
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [9]:
mae = 0.0

for i, img_path in enumerate(img_paths):
    img = Image.open(img_path).convert('RGB')
    img_tensor = transform(img).to(device)

    with h5py.File(img_path.replace('.jpg', '.h5').replace('images', 'ground_truth'), 'r') as gt_file:
        groundtruth = np.asarray(gt_file['density'])

    with torch.no_grad():
        output = model(img_tensor.unsqueeze(0))

    pred_count = output.detach().cpu().sum().item()
    gt_count = float(groundtruth.sum())
    mae += abs(pred_count - gt_count)
    print(i, mae, pred_count, gt_count)

print(mae / len(img_paths))

0 155.7130126953125 643.5357666015625 799.248779296875
1 164.83486938476562 520.0449829101562 510.9231262207031
2 175.04275512695312 366.45672607421875 376.66461181640625
3 295.2594909667969 428.39581298828125 308.1790771484375
4 331.8922424316406 414.2716979980469 377.6389465332031
5 371.1861267089844 97.28715515136719 136.58103942871094
6 374.0262451171875 362.8654479980469 360.02532958984375
7 392.66204833984375 136.08453369140625 117.44873046875
8 442.781005859375 847.4561157226562 797.337158203125
9 519.6467132568359 170.04364013671875 246.9093475341797
10 557.4973602294922 272.5234680175781 234.67282104492188
11 576.1811065673828 147.85818481445312 166.54193115234375
12 627.9569244384766 138.8524169921875 190.62823486328125
6 374.0262451171875 362.8654479980469 360.02532958984375
7 392.66204833984375 136.08453369140625 117.44873046875
8 442.781005859375 847.4561157226562 797.337158203125
9 519.6467132568359 170.04364013671875 246.9093475341797
10 557.4973602294922 272.52346801757